## Imports

In [1]:
import numpy as np
import netCDF4
from scipy import (special, signal, ndimage)

from ipywidgets import (interact)

import bokeh
from bokeh.plotting import (figure, show, output_notebook)
from bokeh.models import (LinearColorMapper, Ticker, ColorBar)
from bokeh.io import (push_notebook)

output_notebook()

Loading BokehJS ...

## Load data

In [56]:
filename = 'data/testcases/sea-ice/sic_MPI-ESM-LR_rcp85_r1i1p1_monthly.nc'
root = netCDF4.Dataset(filename, 'r', format='NETCDF4')

In [57]:
data = root.variables['sic'][:].filled(0.0)
land = root.variables['sic'][0].mask

In [58]:
data.shape

(3540, 220, 256)

## First glimpse

In [59]:
fig2 = figure(x_range=(0, 360), y_range=(-90, 90), plot_width=840, plot_height=640)
img2 = fig2.image(image=[root.variables['sic'][0,::-1,:]], x=0, y=-90, dw=360, dh=180, palette=bokeh.palettes.viridis(255))

def update_fig2(y, m):
    img2.data_source.data['image'] = [root.variables['sic'][y*12+m,::-1,:]]
    push_notebook()

In [60]:
show(fig2, notebook_handle=True)
interact(update_fig2, y=(0, 3540//12 - 1), m=(0,11))

<function __main__.update_fig2>

## Analysis

In [14]:
from hyper_canny import (smooth_sobel, edge_thinning, double_threshold)

In [15]:
x = land.astype('float32').copy()
s = smooth_sobel(x, 5, 0.5)
land_t = edge_thinning(s)
land_c = double_threshold(s, land_t, 50, 100)

In [16]:
def moving_average(a, n=3):
    ret = np.cumsum(a, axis=0, dtype='float32')
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [47]:
x = data[0::12].copy() # moving_average(data[0::12].astype('float32'), n=30)
s = smooth_sobel(x, 5, 2.0)
t = edge_thinning(s)
# s[:,:,:,3] /= s[:,:,:,2]**2

In [48]:
c = double_threshold(s, t, 90., 100.)

In [51]:
fig = figure(x_range=(0, 360), y_range=(-90, 90), plot_width=840, plot_height=640)

color_mapper = LinearColorMapper(palette=bokeh.palettes.viridis(255), low=-5., high=5.0)
color_bar = ColorBar(color_mapper=color_mapper, ticker=Ticker(),
                     label_standoff=12, border_line_color=None, location=(0,0))

ice = np.where(land, -1.0, c[2]).astype('float')
img = fig.image(image=[ice[::-1,:]], x=0, y=-90, dw=360, dh=180, color_mapper=color_mapper)



# fig.add_layout(color_bar, 'right')

state = (1., 1.)

def update_fig(low, high, y):
    global state, c
    if (low, high) != state:
        state = (low, high)
        c = double_threshold(s, t, 1./high, 1./low)
    ice = np.where(land, np.nan, c[y] * s[y,:,:,2] / s[y,:,:,3])
    img.data_source.data['image'] = [ice[::-1,:]]
    push_notebook()

In [55]:
show(fig, notebook_handle=True)
interact(update_fig, low=(0., 10.0), high=(0.0, 10.0), y=(0, c.shape[0]-1))

<function __main__.update_fig>